In [34]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [36]:
df = pd.read_csv('train.csv')

In [37]:
# KMS __________________________________________________

df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

df = df.astype({
    'kms_driven': float,
})
# ENGINE __________________________________________________

df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

df = df.astype({
    'engine_capacity': float,
})

# MAX POWER __________________________________________________
#convert to bhp
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power

df = df.astype({
    'max_power': float,
})
#assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl

        consumo_correto.append(valor)

df['mileage'] = consumo_correto

df = df.astype({
    'mileage': float,
})
df.loc[df["insurance"] == "Third Party insurance","insurance"] = "Third Party"
df.loc[df["insurance"] == "1","insurance"] = "First"
df.loc[df["insurance"] == "2","insurance"] = "Second"
df.loc[df["insurance"] == "Not Available","insurance"] = np.nan
df["insurance"] = df["insurance"].fillna(df["insurance"].mode()[0])

In [38]:
# Transmissao _____________________________________

transmissao = df['full_name']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_nome'] = df['full_name'].map(idf)
df = df.astype({
    'id_nome': float,
})

In [39]:
# Transmissao _____________________________________

transmissao = df['registered_year']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_ano'] = df['registered_year'].map(idf)
df = df.astype({
    'id_ano': float,
})



In [40]:
# Transmissao _____________________________________

transmissao = df['insurance']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_insurance'] = df['insurance'].map(idf)
df = df.astype({
    'id_insurance': float,
})

In [41]:
# Transmissao _____________________________________

transmissao = df['transmission_type']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_transmission_type'] = df['transmission_type'].map(idf)
df = df.astype({
    'id_transmission_type': float,
})

In [42]:
# Substituir valores na coluna 'owner_type'
df['owner_type'] = df['owner_type'].replace({
    'First Owner': '1.0',
    'Second Owner': '2.0',
    'Third Owner': '3.0',
    'Fourth Owner': '4.0',
    'Fifth Owner': '5.0'
})

# Converter para float
df['owner_type'] = df['owner_type'].astype(float)


In [43]:
#Create ID IDF  

# FUEL TYPE __________________________________________________

fuel_types = df['fuel_type']
n_docs = len(df)
idf = {}

# Uniques
unique_fuel_types = fuel_types.drop_duplicates()

print("FUEL TYPE".center(50, "_"))

for fuel_type in unique_fuel_types:
    k = sum(fuel_type == val for val in fuel_types)  
    idf[fuel_type] = np.log10(n_docs / (k + 1))  #+1 to avoid division by zero
    print(f'{fuel_type:>15}: {idf[fuel_type]:>10}')

# Create new column 
df['id_fuel_type'] = df['fuel_type'].map(idf)

df = df.astype({
    'id_fuel_type': float,
})

____________________FUEL TYPE_____________________
         Petrol: 0.18667511225498815
         Diesel: 0.500913650476359
            CNG: 1.5383795956654682
       Electric: 2.4457909564400544
            LPG: 2.7830331247584805


In [44]:
#BODY TYPE __________________

body_type = df['body_type']
n_docs = len(df)
idf = {}


unique_body_type = body_type.drop_duplicates()

for body_type in unique_body_type:
    k = sum(body_type in row for row in df['body_type']) 
    idf[body_type] = np.log10(n_docs / (k + 1))  

for body_type, value in idf.items():
    print(f'{body_type:>15}: {value:>10}')

df['id_body_type'] = df['body_type'].map(idf)

df = df.astype({
    'id_body_type': float,
})

      Hatchback: 0.37457193804008
            SUV: 0.5976018394486557
          Sedan: 0.5626975978643645
            BMW: 3.843730965112092
            MUV: 1.3658864887733337
         Maruti: 2.889488455672767
       Minivans: 2.412367200953105
  Mercedes-Benz: 3.299662920761816
           Cars: 3.299662920761816
          Coupe: 3.144760960776073
         Jaguar: 3.667639706056411
          Wagon: 3.5427009694481106
         Datsun: 3.843730965112092
         Pickup: 3.144760960776073
           Tata: 3.3666097103924297
          Honda: 3.843730965112092
   Convertibles: 3.667639706056411
        Hyundai: 3.843730965112092
       Mahindra: 3.667639706056411
          Volvo: 3.843730965112092
         Toyota: 3.843730965112092
          Isuzu: 3.843730965112092
          Skoda: 3.843730965112092
      Chevrolet: 3.843730965112092


In [45]:
# CITY __________________________________________________

city= df['city']
n_docs = len(df)
idf = {}


unique_cities = city.drop_duplicates()

for city in unique_cities:
    k = sum(city in row for row in df['city'])  
    idf[city] = np.log10(n_docs / (k + 1))  

for city, value in idf.items():
    print(f'{city:>15}: {value:>10}')


df['id_city'] = df['city'].map(idf)
df = df.astype({
    'id_city': float,
})

           Pune: 1.0890004960883384
         Jaipur: 1.289241805108273
         Mumbai: 0.9181610555687159
        Kolkata: 1.1739493499035554
          Delhi: 0.7611250924141936
      Bangalore: 0.8701413416848351
      Hyderabad: 1.0331624358956792
        Chennai: 1.1252292762448177
        Gurgaon: 1.2215169421457968
     Chandigarh: 1.6031817168294924
      Ahmedabad: 1.1198650006685882
        Lucknow: 1.5102836906153416
           Agra: 1.89190792979618


In [46]:
print(df.dtypes)


full_name                object
registered_year          object
engine_capacity         float64
insurance                object
transmission_type        object
kms_driven              float64
owner_type              float64
fuel_type                object
max_power               float64
seats                   float64
mileage                 float64
body_type                object
city                     object
resale_price_Lakh       float64
id_nome                 float64
id_ano                  float64
id_insurance            float64
id_transmission_type    float64
id_fuel_type            float64
id_body_type            float64
id_city                 float64
dtype: object


In [47]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city',
       'resale_price_Lakh', 'id_nome', 'id_ano', 'id_insurance',
       'id_transmission_type', 'id_fuel_type', 'id_body_type', 'id_city'],
      dtype='object')

In [48]:
# Exportar colunas selecionadas para um novo DataFrame
colunas_selecionadas = ['id_nome', 'id_ano', 'engine_capacity', 'id_insurance', 'id_transmission_type','kms_driven', 'owner_type','id_fuel_type', 'max_power', 'seats', 'mileage', 'id_body_type', 'id_city','resale_price_Lakh']

colunas_selecionadas = ['id_nome', 'id_ano', 'engine_capacity', 'id_insurance', 'id_transmission_type', 'kms_driven', 'owner_type', 'id_fuel_type', 'max_power', 'seats', 'mileage', 'id_body_type', 'id_city', 'resale_price_Lakh']

# Verifica nulos nas colunas selecionadas
colunas_com_nulos = df[colunas_selecionadas].columns[df[colunas_selecionadas].isnull().any()].tolist()

if colunas_com_nulos:
    print("Colunas com valores nulos:", colunas_com_nulos)
    # Elimina linhas com valores nulos nessas colunas
    df.dropna(subset=colunas_com_nulos, inplace=True)
    print("Linhas com valores nulos eliminadas.")
else:
    print("Não há valores nulos nas colunas selecionadas.")

df_selecionado = df[colunas_selecionadas]
df_selecionado.to_csv('treino.csv', index=False)

Colunas com valores nulos: ['id_ano', 'engine_capacity', 'kms_driven', 'owner_type', 'max_power', 'seats']
Linhas com valores nulos eliminadas.


In [55]:
#Export
#Not the target
colunas_selecionadas = ['id_nome', 'id_ano','resale_price_Lakh']


if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('CenarioOld.csv', index=False)